In [1]:
import pyscal.crystal_structures as pcs
import pyscal.csystem as pcm
import pyscal.core as pc
from ase import atoms
from ase.io import read, write
from ase.build import bulk
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nx = 10
satoms, box = pcs.make_crystal(structure='bcc', 
    lattice_constant=3.127, 
    repetitions=(nx,nx,nx),)
print(len(satoms["positions"]))
sys = pc.System()
sys.box = box
sys.atoms = satoms


2000


In [3]:
sys.find_neighbors(method="cutoff", 
                   cutoff=3.6)

In [4]:
a = sys.calculate_q(4)

In [5]:
b = sys._calculate_aq([4])

In [8]:
len(b[0])

2000

In [9]:
sys.avg_q4

[0.03636964837266542,
 0.03636964837266542,
 0.03636964837266541,
 0.03636964837266542,
 0.03636964837266541,
 0.0363696483726654,
 0.03636964837266542,
 0.0363696483726654,
 0.03636964837266541,
 0.0363696483726654,
 0.036369648372665396,
 0.036369648372665396,
 0.0363696483726654,
 0.03636964837266541,
 0.03636964837266541,
 0.0363696483726654,
 0.03636964837266541,
 0.0363696483726654,
 0.03636964837266542,
 0.03636964837266542,
 0.03636964837266542,
 0.036369648372665424,
 0.03636964837266542,
 0.03636964837266542,
 0.03636964837266541,
 0.0363696483726654,
 0.03636964837266542,
 0.03636964837266541,
 0.03636964837266541,
 0.0363696483726654,
 0.0363696483726654,
 0.036369648372665396,
 0.0363696483726654,
 0.0363696483726654,
 0.03636964837266542,
 0.0363696483726654,
 0.0363696483726654,
 0.0363696483726654,
 0.0363696483726654,
 0.03636964837266542,
 0.036369648372665424,
 0.03636964837266543,
 0.03636964837266542,
 0.036369648372665424,
 0.03636964837266542,
 0.0363696483726654